# Unbiased Monte Carlo Simulations

Implementation of Henry-Labordère, Tan, and Touzi algorithm

In [1]:
import numpy as np
from unbiased_mc_functions import (
    mu_X, sigma_X,
    compute_gamma, compute_beta,
    run_unbiased_mc, run_unbiased_asian_mc
)

### Parameter Configuration

In [2]:
# Global parameters
dimension = 1
sigma_matrix = 0.5 * np.identity(1)
mu_max = 5
mu_lipschitz = 0.1
T = 1.0
M_param = 4.0
K = 1.0

# Compute optimal beta
gamma = compute_gamma(sigma_matrix, mu_max, dimension, T)
# beta = compute_beta(gamma, mu_lipschitz, T)  # Uncomment for optimal that does not seem to work on our case
beta = 0.15  # Fixed value for stability

print(f"Gamma parameter: {gamma:.4f}")
print(f"Beta parameter: {beta:.4f}")

Gamma parameter: 864.0000
Beta parameter: 0.1500


### European Call Option

In [3]:
# Run unbiased MC for European option
np.random.seed(42)

X_T, X_T_NT, N_T, weights = run_unbiased_mc(
    N_mc=1000000,
    X_0=0.0,
    beta=beta,
    mu_func=mu_X,
    sigma_0=0.5,
    M=M_param,
    T=T
)

# Compute European call payoff
g_X_T = np.maximum(np.exp(X_T) - K, 0)
g_X_T_NT = np.maximum(np.exp(X_T_NT) - K, 0)
indicator = (N_T > 0).astype(float)
payoff = weights * (g_X_T - g_X_T_NT * indicator)

european_price = np.mean(payoff)
european_std = np.std(payoff) / np.sqrt(len(payoff))

print(f"\nEuropean Call Option Price: {european_price:.6f} ± {european_std:.6f}")

c:\Users\jacqu\MonteCarlo\unbiased_mc_functions.py:159: RuntimeWarning: invalid value encountered in divide
  w = (mu_k - mu_prev) * dW_matrix[:, k] / (sigma_0 * dt_matrix[:, k])



European Call Option Price: 0.206018 ± 0.000454


### Asian Call Option (10 timesteps)

In [4]:
# Run unbiased MC for Asian option
np.random.seed(000)

X_paths, X_zero_paths, psi_estimate = run_unbiased_asian_mc(
    N_mc=5000000,
    X_0=0.0,
    beta=beta,
    mu_func=mu_X,
    sigma_0=0.5,
    M=M_param,
    n_timesteps=10,
    T=T
)

asian_price = np.mean(psi_estimate)
asian_std = np.std(psi_estimate) / np.sqrt(len(psi_estimate))

print(f"\nAsian Call Option Price (10 steps): {asian_price:.6f} ± {asian_std:.6f}")

c:\Users\jacqu\MonteCarlo\unbiased_mc_functions.py:245: RuntimeWarning: invalid value encountered in divide
  w = (mu_j - mu_prev) * dW_mat[:, j] / (sigma_0 * dt_mat[:, j])



Asian Call Option Price (10 steps): 0.126573 ± 0.000146


### Comparison Summary

In [5]:
print("="*70)
print("UNBIASED MONTE CARLO RESULTS")
print("="*70)
print(f"European Option: {european_price:.6f} ± {european_std:.6f}")
print(f"Asian Option:    {asian_price:.6f} ± {asian_std:.6f}")
print("="*70)

UNBIASED MONTE CARLO RESULTS
European Option: 0.206018 ± 0.000454
Asian Option:    0.126573 ± 0.000146
